## Objective

Now that I know that my submodels perform a bit better than the model on the entire dataset, I can take advantage of the submodels and find the coefficients that are most associated with response variable.

In [130]:
CATEGORY_GROUPS_IN_QUESTION = [['Pick up Dead Animal', 'Animal Generic Request'],
['Abandoned Vehicles', 'Abandoned Bicycle'],
['Rodent Activity',	'Bed Bugs', 'Mice Infestation - Residential'],
['Sidewalk Repair', 'Sidewalk Repair (Make Safe)'],
['Needle Pickup'],
['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - Excessive  Insufficient'],
['Request for Pothole Repair'],
['Graffiti Removal']]

## Objective

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship and Normalized residuals and imperfect collinearity?

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

from utilities import remove_one_feature

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school, get_vifs

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [26]:
df_orig = transform_school(df_orig)
df_orig.shape

(19742, 40)

## Filterering by `TYPE`

In [17]:
i = 5
print CATEGORY_GROUPS_IN_QUESTION[i]
df_orig = df_orig[df_orig.TYPE.isin(CATEGORY_GROUPS_IN_QUESTION[i])]
df_orig.shape

['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - Excessive  Insufficient']


(19742, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [27]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

(19613, 40)

## Remove `TYPE` col

In [28]:
df_outliers_removed.drop('TYPE', axis=1, inplace=True)

## Choosing columns

In [ ]:
['race_black',
 'race_asian',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'school',
 'school_std_dev',
 'bedroom_std_dev',
 'value',
 'value_std_dev',
 'rent',
 'rent_std_dev',
 'is_description',
 'Source_Constituent Call']

In [147]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'SubmittedPhoto']
cols_census = [
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'housing',
     'housing_std_dev',
     'income',
]
cols_engineered = ['queue_wk', 'queue_wk_open']

In [148]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(19613, 11)

## Removing NAs for cols like `school_std_dev`

In [149]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

7    housing_std_dev
Name: index, dtype: object


In [150]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(19592, 11)

## Dummify

In [151]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['housing']

In [152]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify, chosen_col_i=2)

own is baseline 0 1


In [153]:
df_dummified.shape

(19592, 10)

## Checking for multicollinearity

In [96]:
df_dummified.head(1).T

,905400
COMPLETION_HOURS_LOG_10,0.0124857
SubmittedPhoto,False
poverty_pop_below_poverty_level,0.262473
bedroom,2
queue_wk,12873
queue_wk_open,1
is_description,True


In [42]:
get_vifs(df_dummified.drop(['SubmittedPhoto', 'is_description'], axis=1), 'COMPLETION_HOURS_LOG_10')

Source_Constituent Call            5.664920
Source_Citizens Connect App        5.547692
poverty_pop_w_food_stamps          4.154910
race_black                         3.554310
poverty_pop_below_poverty_level    3.483491
earned_income_per_capita           2.809285
income                             2.457612
race_hispanic                      2.291167
housing_std_dev                    2.281237
school                             2.047018
value_std_dev                      1.816185
poverty_pop_w_ssi                  1.812270
income_std_dev                     1.708968
race_asian                         1.681416
rent_std_dev                       1.614944
school_std_dev                     1.594645
rent                               1.555772
poverty_pop_w_public_assistance    1.535030
bedroom                            1.495423
value                              1.401905
queue_wk_open                      1.261100
queue_wk                           1.235698
bedroom_std_dev                 

## Running model

In [43]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [44]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [45]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

## Use LassoCV to find col subsets

In [46]:
pipe = make_pipeline(StandardScaler(), LassoCV())
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [48]:
params = {'lassocv__alphas': make_alphas(-2, 0)}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=0)
model.fit(X_train, y_train)
pd.DataFrame(model.cv_results_).T.iloc[2:5]

,0,1,2,3,4
mean_test_score,0.161499,0.158137,0.13944,0.0282893,-0.00161932
mean_train_score,0.149065,0.146333,0.131497,0.0302168,0
param_lassocv__alphas,[0.01],[0.03],[0.1],[0.3],[1.0]


In [49]:
model.best_params_

{'lassocv__alphas': [0.01]}

In [50]:
'{} cols go to zero out of {}'.format(
    len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]),
    len(X_train.columns)
)

'16 cols go to zero out of 27'

In [51]:
cols_zero = list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0])
cols_zero

['race_black',
 'race_asian',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'school',
 'school_std_dev',
 'bedroom_std_dev',
 'value',
 'value_std_dev',
 'rent',
 'rent_std_dev',
 'is_description',
 'Source_Constituent Call']

## Use subsetted cols to run lin reg

In [154]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [155]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [156]:
col_list = ' + '.join(df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1).columns)

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [62]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.150
Model:,OLS,Adj. R-squared:,0.149
Method:,Least Squares,F-statistic:,184.6
Date:,"Thu, 23 Feb 2017",Prob (F-statistic):,0.00
Time:,08:43:13,Log-Likelihood:,-28725.
No. Observations:,15673,AIC:,5.748e+04
Df Residuals:,15657,BIC:,5.761e+04
Df Model:,15,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [157]:
y_pred = est.predict(X_test)

In [158]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.13124416335893097

In [159]:
mean_squared_error(y_test, y_pred)**0.5

1.5541458521091562

## Interpreting model

Which features are most associated with completion time?

In [160]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results['coef_abs'] = pd.np.abs(df_results.coef)
df_results.coef_name = df_results.coef_name.map(lambda x: x.strip())
df_results = df_results.sort_values('P>|t|')
df_results.shape

(10, 7)

In [161]:
df_results

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,1.542300e+00,5.700000e-02,26.914,0.000,1.430 1.655,1.542300e+00
1,SubmittedPhoto[T.True],-1.163400e+00,4.400000e-02,-26.258,0.000,-1.250 -1.077,1.163400e+00
4,racehispanic,3.092000e-01,8.000000e-02,3.868,0.000,0.153 0.466,3.092000e-01
6,housingstddev,2.673000e-01,7.300000e-02,3.676,0.000,0.125 0.410,2.673000e-01
8,queuewk,-8.355000e-05,2.510000e-06,-33.241,0.000,-8.85e-05 -7.86e-05,8.355000e-05
9,queuewkopen,2.200000e-03,6.780000e-05,32.156,0.000,0.002 0.002,2.200000e-03
7,income,4.302000e-07,2.280000e-07,1.890,0.059,-1.59e-08 8.76e-07,4.302000e-07
5,raceother,-1.895000e-01,2.300000e-01,-0.824,0.410,-0.640 0.261,1.895000e-01
2,raceblack,4.060000e-02,5.200000e-02,0.789,0.430,-0.060 0.142,4.060000e-02
3,raceasian,8.040000e-02,1.290000e-01,0.622,0.534,-0.173 0.334,8.040000e-02


In [114]:
scores = []

for col in X_train.columns:
    if col != 'Intercept':
        score = remove_one_feature([col], df_dummified)
        scores.append((col, score))
        
sorted(scores, key=lambda x: x[1])[::-1]        

[('SubmittedPhoto', 22.240000000000002),
 ('raceother', 21.532),
 ('incomestddev', 21.319),
 ('PropertyTypeAddress', 21.227),
 ('raceblack', 21.206),
 ('raceasian', 21.201999999999998),
 ('income', 20.637999999999998),
 ('housingstddev', 20.397),
 ('queuewk', 19.708),
 ('queuewkopen', 19.341),
 ('racehispanic', 18.389)]